In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
import csv
import joblib
import pywt
import os
import numpy as np
import cv2 as cv
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, SpatialDropout2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam

In [3]:
IMG_HEIGHT, IMG_WIDTH = 64, 64  # Example size, adjust as necessary

data = []  # To store image data
classes = []  # To store class labels
directory = './drive/MyDrive/LettersDataset'
chars = os.listdir(directory)  # Make sure this gets the right subdirectories
for char in chars:
    for position in ['Beginning', 'End', 'Isolated', 'Middle']:
        path = os.path.join(directory, char, position)
        if os.path.isdir(path):
            listOfFiles = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.png')]  # Adjust extension if necessary
            for filename in listOfFiles:
                img = cv.imread(filename)
                gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                cropped = cv.resize(gray_img, (IMG_HEIGHT, IMG_WIDTH))  # Resize to uniform size
                binary_img = cv.threshold(cropped, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]
                binary_img = binary_img.reshape(IMG_HEIGHT, IMG_WIDTH, 1)  # Reshape to add channel dimension
                data.append(binary_img.astype('float32') / 255.0)  # Normalize and add to data list
                classes.append(char + position)


In [5]:
data = np.array(data)
classes = np.array(classes)

# Assuming `data` is your image array and `classes` is your array of labels
'''indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
classes = classes[indices]'''

# Encode class labels
encoder = LabelEncoder()
encoded_classes = encoder.fit_transform(classes)
categorical_labels = to_categorical(encoded_classes)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data, categorical_labels, test_size=0.2, random_state=42)

In [ ]:
#model 1
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(categorical_labels[0]), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#model 1
#10 epochs
model.fit(X_train, y_train, epochs=10, validation_split=0.2)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

Epoch 1/10
194/194 [==============================] - 49s 255ms/step - loss: 0.5297 - accuracy: 0.7211 - val_loss: 1.3350 - val_accuracy: 0.4625
Epoch 2/10
194/194 [==============================] - 33s 172ms/step - loss: 0.5129 - accuracy: 0.7214 - val_loss: 1.3413 - val_accuracy: 0.4645
Epoch 3/10
194/194 [==============================] - 32s 167ms/step - loss: 0.5107 - accuracy: 0.7301 - val_loss: 1.4199 - val_accuracy: 0.4419
Epoch 4/10
194/194 [==============================] - 36s 187ms/step - loss: 0.5002 - accuracy: 0.7211 - val_loss: 1.5025 - val_accuracy: 0.4490
Epoch 5/10
194/194 [==============================] - 34s 177ms/step - loss: 0.5039 - accuracy: 0.7238 - val_loss: 1.4645 - val_accuracy: 0.4554
Epoch 6/10
194/194 [==============================] - 33s 173ms/step - loss: 0.4948 - accuracy: 0.7271 - val_loss: 1.3980 - val_accuracy: 0.4457
Epoch 7/10
194/194 [==============================] - 34s 175ms/step - loss: 0.4923 - accuracy: 0.7292 - val_loss: 1.4705 - val_ac

In [ ]:
#model 1
#15 epochs
model.fit(X_train, y_train, epochs=15, validation_split=0.2)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

Epoch 1/15
194/194 [==============================] - 33s 172ms/step - loss: 0.7707 - accuracy: 0.6337 - val_loss: 1.0362 - val_accuracy: 0.5252
Epoch 2/15
194/194 [==============================] - 35s 183ms/step - loss: 0.7373 - accuracy: 0.6360 - val_loss: 1.0558 - val_accuracy: 0.5187
Epoch 3/15
194/194 [==============================] - 35s 179ms/step - loss: 0.7036 - accuracy: 0.6441 - val_loss: 1.0665 - val_accuracy: 0.4948
Epoch 4/15
194/194 [==============================] - 35s 179ms/step - loss: 0.6829 - accuracy: 0.6526 - val_loss: 1.0425 - val_accuracy: 0.5019
Epoch 5/15
194/194 [==============================] - 33s 169ms/step - loss: 0.6538 - accuracy: 0.6764 - val_loss: 1.0638 - val_accuracy: 0.5052
Epoch 6/15
194/194 [==============================] - 35s 179ms/step - loss: 0.6359 - accuracy: 0.6720 - val_loss: 1.1238 - val_accuracy: 0.4767
Epoch 7/15
194/194 [==============================] - 36s 185ms/step - loss: 0.6164 - accuracy: 0.6736 - val_loss: 1.1376 - val_ac

In [ ]:
#model 1 with shuffling
#10 epochs
model.fit(X_train, y_train, epochs=10, validation_split=0.2)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")


Epoch 1/10
194/194 [==============================] - 42s 219ms/step - loss: 0.4755 - accuracy: 0.7424 - val_loss: 1.5752 - val_accuracy: 0.4186
Epoch 2/10
194/194 [==============================] - 38s 197ms/step - loss: 0.4597 - accuracy: 0.7492 - val_loss: 1.6154 - val_accuracy: 0.4309
Epoch 3/10
194/194 [==============================] - 33s 169ms/step - loss: 0.4726 - accuracy: 0.7413 - val_loss: 1.5974 - val_accuracy: 0.4406
Epoch 4/10
194/194 [==============================] - 37s 189ms/step - loss: 0.4672 - accuracy: 0.7382 - val_loss: 1.6399 - val_accuracy: 0.4231
Epoch 5/10
194/194 [==============================] - 33s 169ms/step - loss: 0.4582 - accuracy: 0.7511 - val_loss: 1.6913 - val_accuracy: 0.4348
Epoch 6/10
194/194 [==============================] - 33s 172ms/step - loss: 0.4612 - accuracy: 0.7424 - val_loss: 1.6885 - val_accuracy: 0.4154
Epoch 7/10
194/194 [==============================] - 33s 173ms/step - loss: 0.4572 - accuracy: 0.7476 - val_loss: 1.6366 - val_ac

In [ ]:
#model 1 with shuffling
#15 epochs
model.fit(X_train, y_train, epochs=15, validation_split=0.2)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")


Epoch 1/15
194/194 [==============================] - 48s 245ms/step - loss: 0.4219 - accuracy: 0.7616 - val_loss: 2.0160 - val_accuracy: 0.3928
Epoch 2/15
194/194 [==============================] - 33s 167ms/step - loss: 0.4212 - accuracy: 0.7607 - val_loss: 1.9638 - val_accuracy: 0.3960
Epoch 3/15
194/194 [==============================] - 35s 178ms/step - loss: 0.4135 - accuracy: 0.7637 - val_loss: 1.9316 - val_accuracy: 0.4050
Epoch 4/15
194/194 [==============================] - 37s 191ms/step - loss: 0.4137 - accuracy: 0.7581 - val_loss: 1.9991 - val_accuracy: 0.4037
Epoch 5/15
194/194 [==============================] - 33s 171ms/step - loss: 0.4098 - accuracy: 0.7605 - val_loss: 2.0137 - val_accuracy: 0.3966
Epoch 6/15
194/194 [==============================] - 33s 170ms/step - loss: 0.4134 - accuracy: 0.7573 - val_loss: 2.0317 - val_accuracy: 0.4037
Epoch 7/15
194/194 [==============================] - 33s 171ms/step - loss: 0.4105 - accuracy: 0.7718 - val_loss: 1.9394 - val_ac

In [25]:
#model 2 with shuffling
# Initialize the data generator for augmentation
datagen = ImageDataGenerator(
    rotation_range=10,       # Degrees of random rotations
    width_shift_range=0.1,   # Fraction of total width for horizontal shift
    height_shift_range=0.1,  # Fraction of total height for vertical shift
    shear_range=0.1,         # Shearing intensity
    zoom_range=0.1,          # Range for random zoom
    fill_mode='nearest'      # Strategy for filling in newly created pixels
)

# Fit the data generator on your training data
datagen.fit(X_train)
# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(categorical_labels[0]), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

In [ ]:
#model 2 with shuffling
# Train the model using the augmented data generator
model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=15,
    validation_data=(X_test, y_test),
    callbacks=[reduce_lr, early_stopping]
)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")


Epoch 1/15
242/242 [==============================] - 53s 212ms/step - loss: 3.0435 - accuracy: 0.0755 - val_loss: 2.6095 - val_accuracy: 0.1731 - lr: 0.0010
Epoch 2/15
242/242 [==============================] - 51s 211ms/step - loss: 2.1544 - accuracy: 0.2888 - val_loss: 1.2193 - val_accuracy: 0.5307 - lr: 0.0010
Epoch 3/15
242/242 [==============================] - 54s 221ms/step - loss: 1.5355 - accuracy: 0.4270 - val_loss: 1.0984 - val_accuracy: 0.5442 - lr: 0.0010
Epoch 4/15
242/242 [==============================] - 51s 211ms/step - loss: 1.2854 - accuracy: 0.4901 - val_loss: 0.8770 - val_accuracy: 0.5809 - lr: 0.0010
Epoch 5/15
242/242 [==============================] - 51s 210ms/step - loss: 1.1355 - accuracy: 0.5337 - val_loss: 0.8609 - val_accuracy: 0.5959 - lr: 0.0010
Epoch 6/15
242/242 [==============================] - 51s 210ms/step - loss: 1.0787 - accuracy: 0.5342 - val_loss: 0.8557 - val_accuracy: 0.5979 - lr: 0.0010
Epoch 7/15
242/242 [==============================] 

In [17]:
#model 2
# Train the model using the augmented data generator
model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=15,
    validation_data=(X_test, y_test),
    callbacks=[reduce_lr, early_stopping]
)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

Epoch 1/15
242/242 [==============================] - 59s 239ms/step - loss: 3.0003 - accuracy: 0.0890 - val_loss: 2.4675 - val_accuracy: 0.2377 - lr: 0.0010
Epoch 2/15
242/242 [==============================] - 54s 222ms/step - loss: 2.1150 - accuracy: 0.2964 - val_loss: 1.3110 - val_accuracy: 0.4987 - lr: 0.0010
Epoch 3/15
242/242 [==============================] - 56s 232ms/step - loss: 1.4935 - accuracy: 0.4364 - val_loss: 1.1843 - val_accuracy: 0.5085 - lr: 0.0010
Epoch 4/15
242/242 [==============================] - 53s 221ms/step - loss: 1.2798 - accuracy: 0.4935 - val_loss: 0.9323 - val_accuracy: 0.5685 - lr: 0.0010
Epoch 5/15
242/242 [==============================] - 56s 233ms/step - loss: 1.1570 - accuracy: 0.5161 - val_loss: 0.8679 - val_accuracy: 0.5850 - lr: 0.0010
Epoch 6/15
242/242 [==============================] - 56s 232ms/step - loss: 1.0834 - accuracy: 0.5442 - val_loss: 0.9400 - val_accuracy: 0.5669 - lr: 0.0010
Epoch 7/15
242/242 [==============================] 

In [8]:
#model 2 with learning rate changes
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * 0.9

reduce_lr = LearningRateScheduler(scheduler)

model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=50,  # Increased epochs
    validation_data=(X_test, y_test),
    callbacks=[reduce_lr, early_stopping]
)


Epoch 1/50
242/242 [==============================] - 65s 262ms/step - loss: 2.9843 - accuracy: 0.0955 - val_loss: 2.2769 - val_accuracy: 0.2729 - lr: 0.0010
Epoch 2/50
242/242 [==============================] - 58s 241ms/step - loss: 2.1154 - accuracy: 0.2986 - val_loss: 1.3322 - val_accuracy: 0.4956 - lr: 0.0010
Epoch 3/50
242/242 [==============================] - 61s 254ms/step - loss: 1.5895 - accuracy: 0.4204 - val_loss: 1.0562 - val_accuracy: 0.5375 - lr: 0.0010
Epoch 4/50
242/242 [==============================] - 57s 236ms/step - loss: 1.3305 - accuracy: 0.4836 - val_loss: 1.2911 - val_accuracy: 0.5028 - lr: 0.0010
Epoch 5/50
242/242 [==============================] - 55s 228ms/step - loss: 1.2106 - accuracy: 0.5034 - val_loss: 0.8660 - val_accuracy: 0.5922 - lr: 0.0010
Epoch 6/50
242/242 [==============================] - 59s 244ms/step - loss: 1.1166 - accuracy: 0.5395 - val_loss: 0.8572 - val_accuracy: 0.5974 - lr: 0.0010
Epoch 7/50
242/242 [==============================] 

In [26]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * 0.9

reduce_lr = LearningRateScheduler(scheduler)

model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=15,  # Increased epochs
    validation_data=(X_test, y_test),
    callbacks=[reduce_lr, early_stopping]
)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")

Epoch 1/15
242/242 [==============================] - 58s 233ms/step - loss: 2.9664 - accuracy: 0.1012 - val_loss: 2.2705 - val_accuracy: 0.2904 - lr: 0.0010
Epoch 2/15
242/242 [==============================] - 57s 234ms/step - loss: 2.0122 - accuracy: 0.3255 - val_loss: 1.2314 - val_accuracy: 0.5297 - lr: 0.0010
Epoch 3/15
242/242 [==============================] - 54s 222ms/step - loss: 1.4413 - accuracy: 0.4522 - val_loss: 1.1051 - val_accuracy: 0.5349 - lr: 0.0010
Epoch 4/15
242/242 [==============================] - 55s 227ms/step - loss: 1.2121 - accuracy: 0.5097 - val_loss: 0.8875 - val_accuracy: 0.5933 - lr: 0.0010
Epoch 5/15
242/242 [==============================] - 54s 223ms/step - loss: 1.1008 - accuracy: 0.5371 - val_loss: 0.9479 - val_accuracy: 0.5726 - lr: 0.0010
Epoch 6/15
242/242 [==============================] - 56s 232ms/step - loss: 1.0317 - accuracy: 0.5605 - val_loss: 0.8715 - val_accuracy: 0.5840 - lr: 0.0010
Epoch 7/15
242/242 [==============================] 

In [27]:
#model 3

def build_model(conv_layers, dense_neurons, dropout_rate, learning_rate):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))

    for _ in range(conv_layers - 1):  # Adding more convolutional layers based on the parameter
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(2, 2))

    model.add(Flatten())
    model.add(Dense(dense_neurons, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(len(categorical_labels[0]), activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Example of using the build_model function
model = build_model(conv_layers=3, dense_neurons=128, dropout_rate=0.5, learning_rate=0.001)
# Initialize data generator for augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest'
)
datagen.fit(X_train)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Testing different hyperparameters
conv_layers_options = [2, 3]
dense_neurons_options = [128, 256]
dropout_options = [0.5, 0.6]
learning_rates = [0.001, 0.0005]

for conv_layers in conv_layers_options:
    for dense_neurons in dense_neurons_options:
        for dropout_rate in dropout_options:
            for lr in learning_rates:
                model = build_model(conv_layers, dense_neurons, dropout_rate, lr)
                print(f"Training model with {conv_layers} conv layers, {dense_neurons} dense neurons, dropout {dropout_rate}, learning rate {lr}")
                model.fit(
                    datagen.flow(X_train, y_train, batch_size=32),
                    epochs=15,
                    validation_data=(X_test, y_test),
                    callbacks=[reduce_lr, early_stopping]
                )
                loss, accuracy = model.evaluate(X_test, y_test)
                print(f"Test accuracy: {accuracy * 100:.2f}%\n")


Training model with 2 conv layers, 128 dense neurons, dropout 0.5, learning rate 0.001
Epoch 1/15
242/242 [==============================] - 62s 252ms/step - loss: 3.6477 - accuracy: 0.0528 - val_loss: 3.1263 - val_accuracy: 0.0687 - lr: 0.0010
Epoch 2/15
242/242 [==============================] - 62s 254ms/step - loss: 3.2865 - accuracy: 0.0488 - val_loss: 5.0900 - val_accuracy: 0.0672 - lr: 0.0010
Epoch 3/15
242/242 [==============================] - 61s 253ms/step - loss: 3.1670 - accuracy: 0.0519 - val_loss: 90.1802 - val_accuracy: 0.0537 - lr: 0.0010
Epoch 4/15
242/242 [==============================] - 61s 253ms/step - loss: 3.1442 - accuracy: 0.0516 - val_loss: 8.3964 - val_accuracy: 0.0305 - lr: 0.0010
Epoch 5/15
242/242 [==============================] - 62s 257ms/step - loss: 3.1286 - accuracy: 0.0513 - val_loss: 3.1253 - val_accuracy: 0.0543 - lr: 0.0010
Epoch 6/15
242/242 [==============================] - 59s 243ms/step - loss: 3.1341 - accuracy: 0.0506 - val_loss: 3.1236 

In [13]:
#model 4 with shuffle
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 1), kernel_regularizer=l2(0.0001)),
    MaxPooling2D(2, 2),
    SpatialDropout2D(0.1),

    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.0001)),
    MaxPooling2D(2, 2),
    SpatialDropout2D(0.1),

    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.0001)),
    MaxPooling2D(2, 2),
    SpatialDropout2D(0.1),

    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.0001)),
    Dropout(0.9),
    Dense(len(categorical_labels[0]), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

# Train the model using the augmented data generator
model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=10,
    validation_data=(X_test, y_test),
    callbacks=[reduce_lr, early_stopping]
)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy * 100:.2f}%")


Epoch 1/10
242/242 [==============================] - 62s 249ms/step - loss: 3.1621 - accuracy: 0.0472 - val_loss: 3.1490 - val_accuracy: 0.0543 - lr: 0.0010
Epoch 2/10
242/242 [==============================] - 58s 241ms/step - loss: 3.1473 - accuracy: 0.0451 - val_loss: 3.1408 - val_accuracy: 0.0543 - lr: 0.0010
Epoch 3/10
242/242 [==============================] - 58s 238ms/step - loss: 3.1388 - accuracy: 0.0504 - val_loss: 3.1306 - val_accuracy: 0.0543 - lr: 0.0010
Epoch 4/10
242/242 [==============================] - 58s 239ms/step - loss: 3.1239 - accuracy: 0.0495 - val_loss: 3.0733 - val_accuracy: 0.0543 - lr: 0.0010
Epoch 5/10
242/242 [==============================] - 58s 239ms/step - loss: 3.0663 - accuracy: 0.0669 - val_loss: 2.9272 - val_accuracy: 0.0651 - lr: 0.0010
Epoch 6/10
242/242 [==============================] - 59s 244ms/step - loss: 2.9814 - accuracy: 0.0807 - val_loss: 2.8449 - val_accuracy: 0.1096 - lr: 0.0010
Epoch 7/10
242/242 [==============================] 